This is the third assignment for the Coursera course "Advanced Machine Learning and Signal Processing"

Again, please insert to code to your ApacheCouchDB based Cloudant instance below using the "Insert Code" function of Watson Studio (you've done this in Assignment 1 and 2 before)

Done, just execute all cells one after the other and you are done - just note that in the last one you must update your email address (the one you've used for coursera) and obtain a submission token, you get this from the programming assignment directly on coursera.

Please fill in the sections labelled with "###YOUR_CODE_GOES_HERE###"


In [1]:
# The code was removed by Watson Studio for sharing.

Let's create a SparkSession object and put the Cloudant credentials into it

In [2]:
spark = SparkSession\
    .builder\
    .appName("Cloudant Spark SQL Example in Python using temp tables")\
    .config("cloudant.host",credentials_1['custom_url'].split('@')[1])\
    .config("cloudant.username", credentials_1['username'])\
    .config("cloudant.password",credentials_1['password'])\
    .getOrCreate()

Now it’s time to have a look at the recorded sensor data. You should see data similar to the one exemplified below….


In [3]:
df=spark.read.load('shake_classification', "org.apache.bahir.cloudant")

df.createOrReplaceTempView("df")
spark.sql("SELECT * from df").show()


+-----+--------+----+----+----+--------------------+--------------------+
|CLASS|SENSORID|   X|   Y|   Z|                 _id|                _rev|
+-----+--------+----+----+----+--------------------+--------------------+
|    0|asdfghjk|0.56|0.56|0.56|004d76e2b1a0032fb...|1-299af9451269e3f...|
|    0|asdfghjk|0.48|0.48|0.48|004d76e2b1a0032fb...|1-a6293726be13827...|
|    0|asdfghjk|0.52|0.52|0.52|004d76e2b1a0032fb...|1-53df1f5881f49c6...|
|    0|asdfghjk|0.59|0.59|0.59|004d76e2b1a0032fb...|1-60964f5eedcdb4a...|
|    0|asdfghjk|0.49|0.49|0.49|004d76e2b1a0032fb...|1-387ae130f05b80d...|
|    0|asdfghjk|0.39|0.39|0.39|004d76e2b1a0032fb...|1-f7d1e503ce5ee3d...|
|    0|asdfghjk|0.53|0.53|0.53|004d76e2b1a0032fb...|1-67f271865e8425a...|
|    1|asdfghjk|0.56|0.56|0.56|004d76e2b1a0032fb...|1-f30bd5308354359...|
|    1|asdfghjk|0.52|0.52|0.52|004d76e2b1a0032fb...|1-3a9819a97ae307d...|
|    1|asdfghjk|0.43|0.43|0.43|004d76e2b1a0032fb...|1-6210722f6928f53...|
|    1|asdfghjk|0.49|0.49|0.49|004d76e

Let’s check if we have balanced classes – this means that we have roughly the same number of examples for each class we want to predict. This is important for classification but also helpful for clustering

In [4]:
spark.sql("SELECT count(class), class from df group by class").show()

+------------+-----+
|count(class)|class|
+------------+-----+
|          72|    0|
|         187|    1|
+------------+-----+



Let's create a VectorAssembler which consumes columns X, Y and Z and produces a column “features”


In [5]:
from pyspark.ml.feature import VectorAssembler
vectorAssembler = VectorAssembler(inputCols=["X","Y","Z"],
                                  outputCol="features")

Please insatiate a clustering algorithm from the SparkML package and assign it to the clust variable. Here we don’t need to take care of the “CLASS” column since we are in unsupervised learning mode – so let’s pretend to not even have the “CLASS” column for now – but it will become very handy later in assessing the clustering performance. PLEASE NOTE – IN REAL-WORLD SCENARIOS THERE IS NO CLASS COLUMN – THEREFORE YOU CAN’T ASSESS CLASSIFICATION PERFORMANCE USING THIS COLUMN 



In [26]:
from pyspark.ml.clustering import KMeans
help(KMeans)
clust = KMeans(k=20, initMode="random")

Help on class KMeans in module pyspark.ml.clustering:

class KMeans(pyspark.ml.wrapper.JavaEstimator, pyspark.ml.param.shared.HasFeaturesCol, pyspark.ml.param.shared.HasPredictionCol, pyspark.ml.param.shared.HasMaxIter, pyspark.ml.param.shared.HasTol, pyspark.ml.param.shared.HasSeed, pyspark.ml.util.JavaMLWritable, pyspark.ml.util.JavaMLReadable)
 |  K-means clustering with a k-means++ like initialization mode
 |  (the k-means|| algorithm by Bahmani et al).
 |  
 |  >>> from pyspark.ml.linalg import Vectors
 |  >>> data = [(Vectors.dense([0.0, 0.0]),), (Vectors.dense([1.0, 1.0]),),
 |  ...         (Vectors.dense([9.0, 8.0]),), (Vectors.dense([8.0, 9.0]),)]
 |  >>> df = spark.createDataFrame(data, ["features"])
 |  >>> kmeans = KMeans(k=2, seed=1)
 |  >>> model = kmeans.fit(df)
 |  >>> centers = model.clusterCenters()
 |  >>> len(centers)
 |  2
 |  >>> model.computeCost(df)
 |  2.000...
 |  >>> transformed = model.transform(df).select("features", "prediction")
 |  >>> rows = transformed

Let’s train...


In [28]:
from pyspark.ml import Pipeline
pipeline = Pipeline(stages=[vectorAssembler, clust])
model = pipeline.fit(df)

...and evaluate...

In [29]:
prediction = model.transform(df)
prediction.show()

+-----+--------+----+----+----+--------------------+--------------------+----------------+----------+
|CLASS|SENSORID|   X|   Y|   Z|                 _id|                _rev|        features|prediction|
+-----+--------+----+----+----+--------------------+--------------------+----------------+----------+
|    0|asdfghjk|0.56|0.56|0.56|004d76e2b1a0032fb...|1-299af9451269e3f...|[0.56,0.56,0.56]|         6|
|    0|asdfghjk|0.48|0.48|0.48|004d76e2b1a0032fb...|1-a6293726be13827...|[0.48,0.48,0.48]|         0|
|    0|asdfghjk|0.52|0.52|0.52|004d76e2b1a0032fb...|1-53df1f5881f49c6...|[0.52,0.52,0.52]|         4|
|    0|asdfghjk|0.59|0.59|0.59|004d76e2b1a0032fb...|1-60964f5eedcdb4a...|[0.59,0.59,0.59]|         9|
|    0|asdfghjk|0.49|0.49|0.49|004d76e2b1a0032fb...|1-387ae130f05b80d...|[0.49,0.49,0.49]|        10|
|    0|asdfghjk|0.39|0.39|0.39|004d76e2b1a0032fb...|1-f7d1e503ce5ee3d...|[0.39,0.39,0.39]|         1|
|    0|asdfghjk|0.53|0.53|0.53|004d76e2b1a0032fb...|1-67f271865e8425a...|[0.53,0.5

In [30]:
prediction.createOrReplaceTempView('prediction')
spark.sql('''
select max(correct)/max(total) as accuracy from (

    select sum(correct) as correct, count(correct) as total from (
        select case when class != prediction then 1 else 0 end as correct from prediction 
    ) 
    
    union
    
    select sum(correct) as correct, count(correct) as total from (
        select case when class = prediction then 1 else 0 end as correct from prediction 
    ) 
)
''').rdd.map(lambda row: row.accuracy).collect()[0]

0.9266409266409267

If you reached at least 55% of accuracy you are fine to submit your predictions to the grader. Otherwise please experiment with parameters setting to your clustering algorithm, use a different algorithm or just re-record your data and try to obtain. In case you are stuck, please use the Coursera Discussion Forum. Please note again – in a real-world scenario there is no way in doing this – since there is no class label in your data. Please have a look at this further reading on clustering performance evaluation https://en.wikipedia.org/wiki/Cluster_analysis#Evaluation_and_assessment


In [31]:
!rm -Rf a2_m3.json

In [32]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-08-27 10:08:42--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2029 (2.0K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,029       --.-K/s   in 0s      

2018-08-27 10:08:43 (17.6 MB/s) - ‘rklib.py’ saved [2029/2029]



In [33]:
prediction= prediction.repartition(1)
prediction.write.json('a2_m3.json')

In [34]:
!zip -r a2_m3.json.zip a2_m3.json

  adding: a2_m3.json/ (stored 0%)
  adding: a2_m3.json/_SUCCESS (stored 0%)
  adding: a2_m3.json/part-00000-fefa531d-e7ba-46ed-ba61-4eb76eb82863.json (deflated 92%)
  adding: a2_m3.json/._SUCCESS.crc (stored 0%)
  adding: a2_m3.json/.part-00000-fefa531d-e7ba-46ed-ba61-4eb76eb82863.json.crc (stored 0%)


In [35]:
!base64 a2_m3.json.zip > a2_m3.json.zip.base64

In [36]:
from rklib import submit
key = "pPfm62VXEeiJOBL0dhxPkA"
part = "EOTMs"
email = "felipebihaiek@gmail.com"
secret = "yhVlZIARf7MdERs3"

with open('a2_m3.json.zip.base64', 'r') as myfile:
    data=myfile.read()
submit(email, secret, key, part, [part], data)

Submission successful, please check on the coursera grader page for the status
